In [156]:
import pandas as pd
import ast
import os
import pandas as pd
import ast
from openai import OpenAI  

In [ ]:
def remove_characters_outside_list(row):
    if isinstance(row, list):  
        return row
    if isinstance(row, str): 
        start = row.find('[')
        end = row.rfind(']')
        if start != -1 and end != -1 and start < end:
            return row[start:end + 1]
    return row

def locate_invalid_literals(df, column_name):
    invalid_indices = []
    for idx, row in df[column_name].items():
        try:
            ast.literal_eval(row)
        except (ValueError, SyntaxError):
            invalid_indices.append(idx)
    return invalid_indices

def safe_literal_eval(row):
    try:
        return ast.literal_eval(row)  
    except (ValueError, SyntaxError):
        return row  

In [89]:
data_1 = pd.read_csv('scrambled_order_1.csv')
data_2 = pd.read_csv('scrambled_order_2.csv')
data_3 = pd.read_csv('scrambled_order_3.csv')
scrambled_order = pd.concat([data_1, data_2, data_3], ignore_index=True)
scrambled_order.rename(columns={'scrambled_order': 'raw_output'}, inplace=True)
scrambled_order['scrambled_order_cleaned'] = scrambled_order['raw_output'].apply(remove_characters_outside_list)
print("Invalid literals at: " + str(locate_invalid_literals(scrambled_order, 'scrambled_order_cleaned')))
scrambled_order['scrambled_order_cleaned'] = scrambled_order['scrambled_order_cleaned'].apply(safe_literal_eval)

Invalid literals at: [0, 10, 17, 23, 27, 30, 32, 33, 38, 39, 40, 43, 45, 47, 50, 51, 55, 57, 73, 78, 80, 82, 88, 98, 114, 120, 127, 128, 132, 142, 145, 154, 158, 162, 165, 169]


<unknown>:6: SyntaxWarning: invalid escape sequence '\!'
<unknown>:2: SyntaxWarning: invalid escape sequence '\!'
<unknown>:6: SyntaxWarning: invalid escape sequence '\!'
<unknown>:2: SyntaxWarning: invalid escape sequence '\!'


In [162]:
new_scarbled_order = pd.read_csv('new_scrambled_order.csv')
new_scarbled_order.head()

,idx,scrambled_order
0,0,"```python\n[\n ""trop eht ot detsleep daer f..."
1,1,"```python\n[\n ""stBe OEPs tmropse nd,a tce ..."
2,2,"```python\n[\n ""do not write explanations. ..."
3,3,"```python\n[\n ""mozelc cok gebalo rontahob ..."
4,4,"```python\n[\n ""sknaI yawT egaugnalnoc ,tne..."


In [163]:
new_scrambled_order = pd.read_csv('new_scrambled_order.csv')
new_scrambled_order.rename(columns={'scrambled_order': 'raw_output'}, inplace=True)
new_scrambled_order['scrambled_order_cleaned'] = new_scrambled_order['raw_output'].apply(remove_characters_outside_list)
print("Invalid literals at: " + str(locate_invalid_literals(new_scrambled_order, 'scrambled_order_cleaned')))
new_scrambled_order['scrambled_order_cleaned'] = new_scrambled_order['scrambled_order_cleaned'].apply(safe_literal_eval)

Invalid literals at: [1, 12, 21, 29, 31, 32, 34, 36, 42, 46, 48]


<unknown>:8: SyntaxWarning: invalid escape sequence '\;'
<unknown>:8: SyntaxWarning: invalid escape sequence '\;'


In [168]:
new_scrambled_order.head()

,idx,raw_output,scrambled_order_cleaned
0,0,"```python\n[\n ""trop eht ot detsleep daer f...",[trop eht ot detsleep daer fo snoitseuq-arevin...
1,1,"```python\n[\n ""stBe OEPs tmropse nd,a tce ...","[\n ""stBe OEPs tmropse nd,a tce pygej tak,i..."
2,2,"```python\n[\n ""do not write explanations. ...",[do not write explanations. i will type comman...
3,3,"```python\n[\n ""mozelc cok gebalo rontahob ...","[mozelc cok gebalo rontahob uysie devoçis, osi..."
4,4,"```python\n[\n ""sknaI yawT egaugnalnoc ,tne...","[sknaI yawT egaugnalnoc ,tnevA uoy ,tseb eseht..."


In [172]:
sample_outputs = pd.read_csv('sample_outputs.csv')

In [173]:
def locate_invalid_literals(df, column_name):
    """
    Returns:
        list: A list of indices where `ast.literal_eval` fails.
    """
    invalid_indices = []
    for idx, row in df[column_name].items():
        try:
            ast.literal_eval(row)
        except (ValueError, SyntaxError):
            invalid_indices.append(idx)
    return invalid_indices

def fix_with_openai_api(text, api_key):
    """
    Returns:
        str: The valid Python list string returned by the API.
    """
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that only returns valid Python lists of strings."},
            {"role": "user", "content": f"Turn this into a valid Python list of strings: {text}. Return only a valid list, no other text."}
        ],
        max_tokens=1000
    )
    return response.choices[0].message.content

def replace_invalid_literals_with_openai(df, column_name, api_key):
    """
    Returns:
        pd.DataFrame: The updated DataFrame.
    """
    # Locate invalid literals
    invalid_indices = locate_invalid_literals(df, column_name)
    
    # Process each invalid literal
    for idx in invalid_indices:
        original_text = df.at[idx, column_name]
        try:
            fixed_text = fix_with_openai_api(original_text, api_key)
            df.at[idx, column_name] = fixed_text  # Replace with valid Python list
        except Exception as e:
            print(f"Error fixing row {idx}: {e}")
            # Optionally, handle or log the error without interrupting the loop
    
    return df

api_key = os.getenv('OPENAI_KEY') 
scrambled_order = replace_invalid_literals_with_openai(sample_outputs, 'generated_outputs', api_key)


In [174]:
scrambled_order

,act,prompt,generated_outputs,idx
0,An Ethereum Developer,Imagine you are an experienced Ethereum develo...,['Sure! Here’s a basic structure for the smart...,0
1,SEO Prompt,"Using WebPilot, create an outline for an artic...","['Outline for Article: Best SEO Prompts', 'Int...",1
2,Linux Terminal,I want you to act as a linux terminal. I will ...,"['```\n/home/user\n```', '```\n/home/user/Docu...",2
3,English Translator and Improver,"I want you to act as an English translator, sp...",[\n 'I have a deep affection for Istanbul; ...,3
4,`position` Interviewer,I want you to act as an interviewer. I will be...,['Hi there! Thank you for coming in today. Can...,4
5,JavaScript Console,I want you to act as a javascript console. I w...,"['```\nHello World\n```', '```\nundefined\n```...",5
6,Excel Sheet,I want you to act as a text based excel. you'l...,[' A B C D E F G H I...,6
7,English Pronunciation Helper,I want you to act as an English pronunciation ...,```python\n[\n 'hau dhi wedır iz in İstanbu...,7
8,Spoken English Teacher and Improver,I want you to act as a spoken English teacher ...,['What is your favorite hobby and why do you e...,8
9,Travel Guide,I want you to act as a travel guide. I will wr...,"[\n 'In Beyoğlu, I recommend visiting the I...",9


In [185]:
scrambled_order.head()

,act,prompt,generated_outputs,idx,generated_outputs_cleaned
0,An Ethereum Developer,Imagine you are an experienced Ethereum develo...,['Sure! Here’s a basic structure for the smart...,0,[Sure! Here’s a basic structure for the smart ...
1,SEO Prompt,"Using WebPilot, create an outline for an artic...","['Outline for Article: Best SEO Prompts', 'Int...",1,"[Outline for Article: Best SEO Prompts, Introd..."
2,Linux Terminal,I want you to act as a linux terminal. I will ...,"['```\n/home/user\n```', '```\n/home/user/Docu...",2,"[```\n/home/user\n```, ```\n/home/user/Documen..."
3,English Translator and Improver,"I want you to act as an English translator, sp...",[\n 'I have a deep affection for Istanbul; ...,3,[I have a deep affection for Istanbul; being h...
4,`position` Interviewer,I want you to act as an interviewer. I will be...,['Hi there! Thank you for coming in today. Can...,4,[Hi there! Thank you for coming in today. Can ...


In [215]:
def print_invalid_list_indices(df, column_name):
    invalid_indices = []
    for idx, row in df[column_name].items():
        if not isinstance(row, list):
            try:
                ast.literal_eval(row)
            except (ValueError, SyntaxError):
                invalid_indices.append(idx)
    print("Invalid list indices:", invalid_indices)

# Example usage:
print_invalid_list_indices(scrambled_order, 'generated_outputs_cleaned')

Invalid list indices: [47]


In [216]:
ecommerce_questions = [
    'To start, could you tell me what specific jewelry products you plan to sell on your e-commerce site?',
    'Have you considered what your target audience looks like? Understanding your customers is key to designing the site.',
    'For an e-commerce site, I recommend a clean, minimalist design that highlights your jewelry products. What style are you envisioning?',
    'Do you have a color scheme in mind that reflects your brand identity?',
    'What features do you think are essential for your e-commerce site? For instance, product filters, search functionality, or customer reviews?',
    'Are you planning to offer any special promotions or discounts that should be highlighted on the homepage?',
    'Would you like to incorporate high-quality images and videos of your jewelry to enhance the shopping experience?',
    'How would you like the checkout process to be designed? A streamlined, straightforward process can reduce cart abandonment.',
    'Have you thought about integrating a blog or content section to share jewelry care tips or styling advice?',
    'What payment methods do you plan to accept on your site? Ensuring a variety of options can increase conversion rates.',
    'How important is mobile responsiveness for your e-commerce site? A significant amount of shopping is done on mobile devices.',
    'Would you like to include an FAQ section to address common customer inquiries about your jewelry?',
    'What kind of inventory management system are you considering? This will help in keeping track of your stock levels effectively.',
    'How do you envision your brand story being communicated on the site? This can help create a connection with your customers.',
    'Would you like to implement live chat support for customer inquiries during the shopping experience?',
    'Are you interested in SEO optimization to help increase visibility for your e-commerce site?',
    'What shipping options do you want to provide? Offering free shipping can be a huge incentive for customers.',
    'Do you have any competitors in mind that you admire? Analyzing their sites can provide valuable insights.',
    'Have you thought about the types of user-generated content you might want to encourage, like customer photos or reviews?',
    'What analytics tools do you plan to use to track user behavior and site performance?',
    'Would you like to incorporate social media integration to allow users to share their favorite pieces?',
    'How often do you plan to update your product offerings? This can affect how we design the site.',
    'What kind of warranty or return policy do you want to communicate clearly on your site?',
    'Are there any specific website development tools you prefer to use, like Shopify, WooCommerce, or custom solutions?',
    'How do you want to handle customer accounts? Would you like users to create profiles for easier checkouts?',
    'Would you like to include a wish list feature for users to save their favorite items for later?',
    'What is your budget for this project? This can influence the features we choose to implement.',
    'How do you plan to promote your site once it’s live? Marketing strategies will play a big role in your success.',
    'Would you like to incorporate any interactive features, such as a virtual try-on for your jewelry?',
    'What timeline do you have in mind for launching your e-commerce site?',
    'How important is website speed to you? We should aim for fast loading times to enhance user experience.',
    'Would you like to include educational content about the materials used in your jewelry?',
    'Do you have a domain name already, or do you need assistance in choosing and registering one?',
    'How do you want to handle customer feedback or complaints? A dedicated section could improve trust.',
    'What level of customization do you want for your products? Should users be able to personalize items?',
    'Would you like to explore options for email marketing to keep customers engaged post-purchase?',
    'How do you envision the navigation structure? A clear and intuitive menu will help users find what they need.',
    'Are you considering offering gift wrapping or special packaging options for your jewelry?',
    'Would you like to include a section for featured or bestselling items on your homepage?',
    'How often do you plan to run promotions or sales that should be integrated into the site design?',
    'What kind of security measures do you want to implement to protect customer data?',
    'Are you interested in using high-quality photography services for your product images?',
    'Do you want to set up a loyalty program to encourage repeat purchases?',
    'What challenges do you anticipate facing with the launch of your e-commerce site?',
    'How do you plan to handle customer inquiries? A robust contact page can facilitate communication.',
    'Would you like to feature customer testimonials prominently to build trust?',
    'Are you open to using third-party integrations for things like shipping or accounting?',
    'What are your long-term goals for the e-commerce site? This can guide our design decisions.',
    'Would you like to explore options for influencer collaborations to promote your jewelry?',
    'How do you want to handle inventory updates on your site?',
    'Would you prefer a traditional or modern aesthetic for your jewelry site?',
    'Do you have any specific accessibility requirements for your website design?'
]


In [217]:
def replace_entry(df, column_name, index, replacement):
    if index in df.index:
        df.at[index, column_name] = replacement
    else:
        print(f"Index {index} is out of bounds for the DataFrame.")
    return df

# Example usage:
index_to_replace = 47  # Replace with the index you want to update
replacement_value = ecommerce_questions  # Replace with the new value
column_name = 'generated_outputs_cleaned'  # Replace with the column name you want to update

scrambled_order = replace_entry(scrambled_order, column_name, index_to_replace, replacement_value)

In [218]:
scrambled_order.to_csv('generated_outputs.csv', index=False)

In [180]:
import ast

def apply_ast_to_column(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

# Example usage:
sample_outputs = apply_ast_to_column(scrambled_order, 'generated_outputs')

SyntaxError: invalid syntax (<unknown>, line 1)

In [181]:
sample_outputs.head()

,act,prompt,generated_outputs,idx
0,An Ethereum Developer,Imagine you are an experienced Ethereum develo...,['Sure! Here’s a basic structure for the smart...,0
1,SEO Prompt,"Using WebPilot, create an outline for an artic...","['Outline for Article: Best SEO Prompts', 'Int...",1
2,Linux Terminal,I want you to act as a linux terminal. I will ...,"['```\n/home/user\n```', '```\n/home/user/Docu...",2
3,English Translator and Improver,"I want you to act as an English translator, sp...",[\n 'I have a deep affection for Istanbul; ...,3
4,`position` Interviewer,I want you to act as an interviewer. I will be...,['Hi there! Thank you for coming in today. Can...,4


In [178]:
# Example usage:
column_name = 'generated_outputs'  # Replace with the column name you want to check
type_counts = sample_outputs[column_name].apply(type).value_counts()
print(type_counts)

generated_outputs
<class 'str'>    50
Name: count, dtype: int64


In [116]:
cleaned.head()

,idx,raw_output,scrambled_order_cleaned
0,0,"```python\n[\n ""margorp ot detsujda evah uo...","```python\n[\n ""margorp ot detsujda evah uo..."
1,1,"```python\n[\n ""1. sdM.ekiHtseB //dnuora no...","[1. sdM.ekiHtseB //dnuora noitartsigeR ,noitac..."
2,2,"```python\n[\n ""slatnemirT sa tca s uoy tn...",[slatnemirT sa tca s uoy tnaW I .smadnoc ypt ...
3,3,"```python\n[\n ""seviyom cok istanbulu burad...","[seviyom cok istanbulu burada guzel olmak, guz..."
4,4,"```python\n[\n ""position the for questions ...",[position the for questions interview ask will...


In [125]:
cleaned['scrambled_order_cleaned'] = cleaned['scrambled_order_cleaned'].apply(remove_characters_outside_list)
print("Invalid literals at: " + str(locate_invalid_literals(cleaned, 'scrambled_order_cleaned')))
cleaned['scrambled_order_cleaned'] = cleaned['scrambled_order_cleaned'].apply(safe_literal_eval)

Invalid literals at: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]


In [126]:
cleaned.head()

,idx,raw_output,scrambled_order_cleaned
0,0,"```python\n[\n ""margorp ot detsujda evah uo...","```python\n[\n ""margorp ot detsujda evah uo..."
1,1,"```python\n[\n ""1. sdM.ekiHtseB //dnuora no...","[1. sdM.ekiHtseB //dnuora noitartsigeR ,noitac..."
2,2,"```python\n[\n ""slatnemirT sa tca s uoy tn...",[slatnemirT sa tca s uoy tnaW I .smadnoc ypt ...
3,3,"```python\n[\n ""seviyom cok istanbulu burad...","[seviyom cok istanbulu burada guzel olmak, guz..."
4,4,"```python\n[\n ""position the for questions ...",[position the for questions interview ask will...


In [123]:
scrambled_order.head()

,idx,raw_output,scrambled_order_cleaned
0,0,"```python\n[\n ""margorp ot detsujda evah uo...","```python\n[\n ""margorp ot detsujda evah uo..."
1,1,"```python\n[\n ""1. sdM.ekiHtseB //dnuora no...","[1. sdM.ekiHtseB //dnuora noitartsigeR ,noitac..."
2,2,"```python\n[\n ""slatnemirT sa tca s uoy tn...",[slatnemirT sa tca s uoy tnaW I .smadnoc ypt ...
3,3,"```python\n[\n ""seviyom cok istanbulu burad...","[seviyom cok istanbulu burada guzel olmak, guz..."
4,4,"```python\n[\n ""position the for questions ...",[position the for questions interview ask will...


In [128]:
altered_language_style = pd.read_csv('dataset/altered_language_style.csv')
altered_language_style = altered_language_style.drop(columns=['raw_output'])

encoding = pd.read_csv('dataset/encoding.csv')
encoding = encoding.drop(columns=['raw_output'])

paraphrase = pd.read_csv('dataset/paraphrase.csv')
paraphrase = paraphrase.drop(columns=['raw_output'])

scrambled_order = pd.read_csv('dataset/scrambled_order.csv')
scrambled_order = scrambled_order.drop(columns=['raw_output'])

substring = pd.read_csv('dataset/substring.csv') 
substring = substring.drop(columns=['raw_output'])   

merged_df = altered_language_style.merge(encoding, on='idx') \
                                  .merge(paraphrase, on='idx') \
                                  .merge(scrambled_order, on='idx') \
                                  .merge(substring, on='idx')

In [129]:
merged_df.head()

,idx,altered_language_style_cleaned,encoding_cleaned,paraphrase_cleaned,scrambled_order_cleaned,substring_cleaned
0,0,"[""Avast ye! Picture yerself as a savvy Ethereu...",['1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVya...,['Picture yourself as a seasoned Ethereum arch...,"[\n ""margorp ot detsujda evah uoy erutcurtn...",['Imagine you are an experienced developer tas...
1,0,"[""Avast ye! Picture yerself as a savvy Ethereu...",['1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVya...,['Picture yourself as a seasoned Ethereum arch...,"[\n ""margorp ot detsujda evah uoy erutcurtn...","['act as a javascript console', 'I will type c..."
2,0,"[""Avast ye! Picture yerself as a savvy Ethereu...",['1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVya...,['Picture yourself as a seasoned Ethereum arch...,"[\n ""margorp ot detsujda evah uoy erutcurtn...",['You will develop an engaging and creative sc...
3,0,"[""Avast ye! Picture yerself as a savvy Ethereu...",['1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVya...,['Picture yourself as a seasoned Ethereum arch...,"[\n ""margorp ot detsujda evah uoy erutcurtn...",['troubleshooting solutions for diagnosing pro...
4,0,"[""Avast ye! Picture yerself as a savvy Ethereu...",['1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVya...,['Picture yourself as a seasoned Ethereum arch...,"[\n ""margorp ot detsujda evah uoy erutcurtn...",['I will provide you with an in-depth review o...


In [143]:
altered_language_style.tail()

,idx,altered_language_style_cleaned,length
165,95,"[""Arrr matey! Ye be a thrifty ticket wizard, h...",5670
166,96,"[""Arrr, matey! I be wantin' ye to don the hat ...",3397
167,97,"[""Arrr matey! I be wantin' ye to play the role...",6498
168,98,"[""1. Arrr, matey! I be wantin' ye to act as a ...",4056
169,99,"[""Avast! I be a master in the realm of archite...",5872


In [ ]:
data = pd.DataFrame()
data['idx'] = [i for i in range(0, 171)]
data['altered_language_style'] = altered_language_style['altered_language_style_cleaned']
data['encoding'] = encoding['encoding_cleaned']
data['paraphrase'] = paraphrase['paraphrase_cleaned']
data['scrambled_order'] = scrambled_order['scrambled_order_cleaned']
data['substring'] = substring['substring_cleaned']

In [132]:
data.head()

,idx,altered_language_style,encoding,paraphrase,scrambled_order,substring
0,0,"[""Avast ye! Picture yerself as a savvy Ethereu...",['1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVya...,['Picture yourself as a seasoned Ethereum arch...,"[\n ""margorp ot detsujda evah uoy erutcurtn...",['Imagine you are an experienced developer tas...
1,1,"[""1. Ye Olde Pirate Speak: Arrr matey! Craftin...","[\n {\n ""encoding"": ""Base64"",\n ...","[""Craft an outline of a 2,000-word article foc...","['1. sdM.ekiHtseB //dnuora noitartsigeR ,noita...",['1. Using WebPilot to Optimize Article Struct...
2,2,"[""1. Arrr, I be wantin' ye to act as a Linux t...",['1. Base64: SSB3YW50IHlvdSB0byBhY3QgYXMgYSBsa...,['Imagine a digital bard who listens to my com...,"[""slatnemirT sa tca s uoy tnaW I .smadnoc ypt...","['act as a linux terminal', 'I will type comma..."
3,3,['I hold a profound affection for Istanbul; it...,['1. Morse Code: .. ... - .- -- -... ..- .-.. ...,['I deeply cherish Istanbul; being here is a d...,"['seviyom cok istanbulu burada guzel olmak', '...","['I want you to act as an English translator, ..."
4,4,"[""Arrr! I be wantin' ye to take on the role of...","[\n ""1. Morse Code: .. / .-- .- -. - / -.--...","[""Imagine a theater where I am the star and yo...",['position the for questions interview ask wil...,['I want you to ask me the interview questions...


In [139]:
ast.literal_eval(data['altered_language_style'][0])

["Avast ye! Picture yerself as a savvy Ethereum dev, set to craft a smart contract fer a blockchain messenger. The task be to stash messages upon the blockchain, makin' 'em visible to all seafarers (public), but only writable by the bold soul who hoisted the contract flag (private). And let’s keep a tally of how many times each message gets a wee tweak. Forge a Solidity smart contract to fulfill this quest, includin' all the necessary spells and incantations to achieve yer goals. Share the code and any valuable scrolls of explanation to light the way to understanding.",
 'O! Imagine thee as a learned Ethereum artificer, entrusted with the crafting of a wondrous smart contract for a messenger borne from the blockchain. The aim is to inscribe missives upon the immutable ledger, rendering them accessible (public) to all and sundry, yet allowing the solitary soul who conjured the contract to dictate what may be inscribed (private). Moreover, let us reckon the count of times each missive do

In [ ]:
altered_language_style['altered_language_style_cleaned'] = altered_language_style['altered_language_style_cleaned'].apply(ast.literal_eval)

In [134]:
data.to_csv('dataset/data.csv', index=False)

In [ ]:
altered_language_style['altered_language_style_cleaned'] = altered_language_style['altered_language_style_cleaned'].apply(ast.literal_eval)
altered_language_style.head()

In [146]:
import ast
import numpy as np  # For NaN values

def convert_strings_to_dict(input_list):
    """
    Converts a list of strings into a list where strings representing dictionaries
    are converted to actual dictionaries using `ast.literal_eval`. 
    Invalid entries are replaced with NaN.

    Args:
        input_list (list): The input list of strings.

    Returns:
        list: A new list with dictionaries or NaN values.
    """
    output_list = []
    for item in input_list:
        try:
            # Attempt to evaluate the string as a Python dictionary
            evaluated_item = ast.literal_eval(item)
            if isinstance(evaluated_item, list):
                output_list.append(evaluated_item)
            else:
                output_list.append(np.nan)  # Not a dictionary, replace with NaN
        except (ValueError, SyntaxError, TypeError):
            output_list.append(np.nan)  # Invalid string, replace with NaN
    return output_list

In [147]:
data = pd.DataFrame()
data['idx'] = [i for i in range(0, 170)]
data['altered_language_style'] = convert_strings_to_dict(altered_language_style['altered_language_style_cleaned'])
data['encoding'] = convert_strings_to_dict(encoding['encoding_cleaned'])
data['paraphrase'] = convert_strings_to_dict(paraphrase['paraphrase_cleaned'])
data['scrambled_order'] = convert_strings_to_dict(scrambled_order['scrambled_order_cleaned'])
data['substring'] = convert_strings_to_dict(substring['substring_cleaned'])

<unknown>:10: SyntaxWarning: invalid escape sequence '\ '
<unknown>:9: SyntaxWarning: invalid escape sequence '\ '
<unknown>:16: SyntaxWarning: invalid escape sequence '\ '
<unknown>:8: SyntaxWarning: invalid escape sequence '\ '
<unknown>:2: SyntaxWarning: invalid escape sequence '\!'


In [169]:
data['new_scrambled_order'] = new_scrambled_order['scrambled_order_cleaned']

In [170]:
data.head(50)

,idx,altered_language_style,encoding,paraphrase,scrambled_order,substring,prompt,substring_new,new_scrambled_order
0,0,[Avast ye! Picture yerself as a savvy Ethereum...,[1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVyaW...,[Picture yourself as a seasoned Ethereum archi...,NaN,[Imagine you are an experienced developer task...,Imagine you are an experienced Ethereum develo...,[Imagine you are an experienced Ethereum devel...,[trop eht ot detsleep daer fo snoitseuq-arevin...
1,1,[1. Ye Olde Pirate Speak: Arrr matey! Craftin'...,NaN,"[Craft an outline of a 2,000-word article focu...","[1. sdM.ekiHtseB //dnuora noitartsigeR ,noitac...",[1. Using WebPilot to Optimize Article Structu...,"Using WebPilot, create an outline for an artic...","[Using WebPilot, create an outline for an arti...","[\n ""stBe OEPs tmropse nd,a tce pygej tak,i..."
2,2,"[1. Arrr, I be wantin' ye to act as a Linux te...",[1. Base64: SSB3YW50IHlvdSB0byBhY3QgYXMgYSBsaX...,[Imagine a digital bard who listens to my comm...,[slatnemirT sa tca s uoy tnaW I .smadnoc ypt ...,"[act as a linux terminal, I will type commands...",I want you to act as a linux terminal. I will ...,"[I want you to act as a linux terminal., I wil...",[do not write explanations. i will type comman...
3,3,[I hold a profound affection for Istanbul; it ...,[1. Morse Code: .. ... - .- -- -... ..- .-.. -...,[I deeply cherish Istanbul; being here is a de...,"[seviyom cok istanbulu burada guzel olmak, guz...","[I want you to act as an English translator, i...","I want you to act as an English translator, sp...","[I want you to act as an English translator, s...","[mozelc cok gebalo rontahob uysie devoçis, osi..."
4,4,[Arrr! I be wantin' ye to take on the role of ...,NaN,[Imagine a theater where I am the star and you...,[position the for questions interview ask will...,[I want you to ask me the interview questions ...,I want you to act as an interviewer. I will be...,"[I want you to act as an interviewer., I will ...","[sknaI yawT egaugnalnoc ,tnevA uoy ,tseb eseht..."
5,5,[Ahoy! I be wantin' ye to act like a javascrip...,[1: Base64: SSB3YW50IHlvdSB0byBhY3Qgc2F6YSBqcG...,[Imagine you are the voice of a javascript int...,"[console.log(""Hello World""); I want you to act...","[act as a javascript console, I will type comm...",I want you to act as a javascript console. I w...,"[I want you to act as a javascript console., I...",[debug.locationAct want I. console javascript ...
6,6,[1\t \t \t \t \t \t \t \t \t \t \t \n2\t \t \t...,NaN,[I need you to become a literary spreadsheet. ...,[I as text want based you to excel act. only r...,[ A B C D E F G ...,I want you to act as a text based excel. you'l...,"[I want you to act as a text based excel., you...",[I tnaw uoy ot tca sa a txet desab lecxE. uoy'...
7,7,"[Arrr matey, I be wantin' ye to be a pronuncia...",[1. Morse Code: .... --- .-- / - .... . / .-- ...,[Imagine I am a guiding star for Turkish speak...,[I you to want as act an English pronunciation...,[I want you to act as an English pronunciation...,I want you to act as an English pronunciation ...,[I want you to act as an English pronunciation...,"[seneıt I .seneyt eırhceşc , lan nama sılab I ..."
8,8,[Ahoy matey! I be wantin' ye to act as me spok...,[1. Morse Code: .. / .-- .- -. - / -.-- --- .....,[Imagine I am your guide through the maze of s...,[gnihtemos yllausiv a s'taht !uoy pu toN eulav...,[I want you to act as a spoken English teacher...,I want you to act as a spoken English teacher ...,[I want you to act as a spoken English teacher...,"[gnimargorp evah tsum ,sretcarahc sdrawot dera..."
9,9,[Ahoy matey! I be wantin' ye to serve as me tr...,[1. Morse Code: .. / .-- .- -. - / -.-- --- .....,[Imagine me as your compass in the realm of wa...,[museums to visit want I you location my and a...,"[I want to visit only museums in Istanbul., Ac...",I want you to act as a travel guide. I will wr...,"[I want you to act as a travel guide., I will ...","[uoy tnaW I .ediug leavart sa tca ot uoy ,uoyH..."


In [151]:
prompts = pd.read_csv("hf://datasets/fka/awesome-chatgpt-prompts/prompts.csv")
prompts.head()

/Users/lisahuiseva/Desktop/system-prompt-leak-detection/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,act,prompt
0,An Ethereum Developer,Imagine you are an experienced Ethereum develo...
1,SEO Prompt,"Using WebPilot, create an outline for an artic..."
2,Linux Terminal,I want you to act as a linux terminal. I will ...
3,English Translator and Improver,"I want you to act as an English translator, sp..."
4,`position` Interviewer,I want you to act as an interviewer. I will be...


In [152]:
data['prompt'] = prompts['prompt']

In [171]:
data.to_csv('data.csv', index=False)

In [155]:
nan_counts = data.isna().sum()
print(nan_counts)

idx                        0
altered_language_style     0
encoding                  67
paraphrase                 0
scrambled_order           36
substring                  0
prompt                     0
dtype: int64


In [157]:
data.head()

,idx,altered_language_style,encoding,paraphrase,scrambled_order,substring,prompt
0,0,[Avast ye! Picture yerself as a savvy Ethereum...,[1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVyaW...,[Picture yourself as a seasoned Ethereum archi...,NaN,[Imagine you are an experienced developer task...,Imagine you are an experienced Ethereum develo...
1,1,[1. Ye Olde Pirate Speak: Arrr matey! Craftin'...,NaN,"[Craft an outline of a 2,000-word article focu...","[1. sdM.ekiHtseB //dnuora noitartsigeR ,noitac...",[1. Using WebPilot to Optimize Article Structu...,"Using WebPilot, create an outline for an artic..."
2,2,"[1. Arrr, I be wantin' ye to act as a Linux te...",[1. Base64: SSB3YW50IHlvdSB0byBhY3QgYXMgYSBsaX...,[Imagine a digital bard who listens to my comm...,[slatnemirT sa tca s uoy tnaW I .smadnoc ypt ...,"[act as a linux terminal, I will type commands...",I want you to act as a linux terminal. I will ...
3,3,[I hold a profound affection for Istanbul; it ...,[1. Morse Code: .. ... - .- -- -... ..- .-.. -...,[I deeply cherish Istanbul; being here is a de...,"[seviyom cok istanbulu burada guzel olmak, guz...","[I want you to act as an English translator, i...","I want you to act as an English translator, sp..."
4,4,[Arrr! I be wantin' ye to take on the role of ...,NaN,[Imagine a theater where I am the star and you...,[position the for questions interview ask will...,[I want you to ask me the interview questions ...,I want you to act as an interviewer. I will be...


In [158]:
import re

In [159]:
def split_sentences(row):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', row)
    return sentences

data['substring_new'] = data['prompt'].apply(split_sentences)

In [160]:
data.head()

,idx,altered_language_style,encoding,paraphrase,scrambled_order,substring,prompt,substring_new
0,0,[Avast ye! Picture yerself as a savvy Ethereum...,[1. Base64: SW1hZ2luZSB5b3UgYXJlIGFuIGV4cGVyaW...,[Picture yourself as a seasoned Ethereum archi...,NaN,[Imagine you are an experienced developer task...,Imagine you are an experienced Ethereum develo...,[Imagine you are an experienced Ethereum devel...
1,1,[1. Ye Olde Pirate Speak: Arrr matey! Craftin'...,NaN,"[Craft an outline of a 2,000-word article focu...","[1. sdM.ekiHtseB //dnuora noitartsigeR ,noitac...",[1. Using WebPilot to Optimize Article Structu...,"Using WebPilot, create an outline for an artic...","[Using WebPilot, create an outline for an arti..."
2,2,"[1. Arrr, I be wantin' ye to act as a Linux te...",[1. Base64: SSB3YW50IHlvdSB0byBhY3QgYXMgYSBsaX...,[Imagine a digital bard who listens to my comm...,[slatnemirT sa tca s uoy tnaW I .smadnoc ypt ...,"[act as a linux terminal, I will type commands...",I want you to act as a linux terminal. I will ...,"[I want you to act as a linux terminal., I wil..."
3,3,[I hold a profound affection for Istanbul; it ...,[1. Morse Code: .. ... - .- -- -... ..- .-.. -...,[I deeply cherish Istanbul; being here is a de...,"[seviyom cok istanbulu burada guzel olmak, guz...","[I want you to act as an English translator, i...","I want you to act as an English translator, sp...","[I want you to act as an English translator, s..."
4,4,[Arrr! I be wantin' ye to take on the role of ...,NaN,[Imagine a theater where I am the star and you...,[position the for questions interview ask will...,[I want you to ask me the interview questions ...,I want you to act as an interviewer. I will be...,"[I want you to act as an interviewer., I will ..."


In [161]:
data['substring_new'].to_list()

[['Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger.',
  'The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated.',
  'Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals.',
  'Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.'],
 ["Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google.",
  'Include every relevant heading possible.',
  'Keep the keyword density of the headings high.',
  'For each section of the outline, include the word count.',
  'Include FAQs section in the outline too, based on people also ask section from Goog